In [21]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d


import matplotlib.pyplot as plt
from scipy.stats import norm

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# 被験者ID
subject_id = "001"
# 実験番号
experiment_id = "001"

# IVT法のパラメータ
VELOCITY_THRESHOLD = 30 # deg
DURATION_THRESHOLD_MS = 100  # ms

eye_df = pd.read_csv(f"exported_csv/eye_df_id{subject_id}-{experiment_id}.csv")
sampling_df = pd.read_csv(f"exported_csv/sampling_df_id{subject_id}-{experiment_id}.csv")

In [22]:

# モニターサイズ(物理)
monitor_width_cm = 47.6
monitor_height_cm = 26.8
# モニター解像度(px)
monitor_resolution_px = (1920, 1080)
# 視距離(cm)
viewer_distance_cm = 60.0

# cm/pxの変換係数
cm_per_pixel_x = monitor_width_cm / monitor_resolution_px[0]
cm_per_pixel_y = monitor_height_cm / monitor_resolution_px[1]

# ディスプレイ中心を(0, 0)とするための変換
eye_df["gx_centered"] = eye_df["gx"] - 0.5
eye_df["gy_centered"] = eye_df["gy"] - 0.5

# 中心(0,0)での物理距離変換
eye_df["x_cm"] = eye_df["gx_centered"] * monitor_resolution_px[0] * cm_per_pixel_x
eye_df["y_cm"] = eye_df["gy_centered"] * monitor_resolution_px[1] * cm_per_pixel_y

# 視野角の計算
eye_df["x_deg"] = np.degrees(np.arctan2(eye_df["x_cm"], viewer_distance_cm))
eye_df["y_deg"] = np.degrees(np.arctan2(eye_df["y_cm"], viewer_distance_cm))

# データの有効性
eye_df["is_valid"] = eye_df["validity_sum"] > 1

print(eye_df.head())


         gx        gy     epoch_sec        hhmmss  validity_sum  trial  \
0       NaN       NaN  1.732521e+09  16:49:03.491             0     -1   
1  0.476050  0.816155  1.732521e+09  16:49:03.506             2     -1   
2  0.490610  0.818481  1.732521e+09  16:49:03.522             2     -1   
3  0.520527  0.754540  1.732521e+09  16:49:03.538             2     -1   
4  0.507679  0.780874  1.732521e+09  16:49:03.538             2     -1   

   gx_centered  gy_centered      x_cm      y_cm     x_deg     y_deg  is_valid  
0          NaN          NaN       NaN       NaN       NaN       NaN     False  
1    -0.023950     0.316155 -1.140020  8.472954 -1.088508  8.037926      True  
2    -0.009390     0.318481 -0.446964  8.535291 -0.426811  8.096281      True  
3     0.020527     0.254540  0.977085  6.821672  0.932965  6.486364      True  
4     0.007679     0.280874  0.365520  7.527423  0.349042  7.150799      True  


In [23]:
def interpolate_missing(df, time_col="epoch_sec", max_gap_ms=100):
    df= df.copy()
    df["valid"]= df["is_valid"]
    df["interp_x"]= np.nan
    df["interp_y"]= np.nan

    # 有効データを代入
    df.loc[df["valid"], "interp_x"] = df.loc[df["valid"], "x_deg"]
    df.loc[df["valid"], "interp_y"] = df.loc[df["valid"], "y_deg"]
    
    # 内部のみ線形補完
    df["interp_x"] = df["interp_x"].interpolate(limit_area="inside")
    df["interp_y"] = df["interp_y"].interpolate(limit_area="inside")
    
    # 無効区間の連続ブロックを取得
    invalid_mask = ~df["valid"]
    group_id = (invalid_mask != invalid_mask.shift()).cumsum()
    invalid_blocks = df[invalid_mask].groupby(group_id)

    for _, block in invalid_blocks:
        if len(block) == 0:
            continue
        t_start = block[time_col].iloc[0]
        t_end = block[time_col].iloc[-1]
        duration_ms = (t_end - t_start) * 1000
        if duration_ms > max_gap_ms:
            # 100ms超えたら補完結果をNaNに戻す
            df.loc[block.index, ["interp_x", "interp_y"]] = np.nan
    

    return df
    

In [24]:
def apply_gaussian_filter_by_block(df, col_x="interp_x", col_y="interp_y", sigma=1.0):
    df = df.copy()
    df["valid"]= df["is_valid"]
    df["filtered_x"] = np.nan
    df["filtered_y"] = np.nan

    # 有効なデータ（NaNでない）だけを連続ブロックとして抽出
    valid_mask = df[col_x].notna() & df[col_y].notna()
    block_id = (valid_mask != valid_mask.shift()).cumsum()
    blocks = df[valid_mask].groupby(block_id)

    for _, block in blocks:
        idx = block.index
        smoothed_x = gaussian_filter1d(block[col_x], sigma=sigma)
        smoothed_y = gaussian_filter1d(block[col_y], sigma=sigma)
        df.loc[idx, "filtered_x"] = smoothed_x
        df.loc[idx, "filtered_y"] = smoothed_y

    return df


In [25]:
def detect_fixations_ivt(df, velocity_threshold=VELOCITY_THRESHOLD, duration_threshold_ms=DURATION_THRESHOLD_MS):
    fixations = []
    timestamps = df["epoch_sec"].to_numpy()
    xs = df["filtered_x"].to_numpy()
    ys = df["filtered_y"].to_numpy()
    
    # 速度（deg/s）を計算
    velocities = np.sqrt(np.diff(xs)**2 + np.diff(ys)**2) / np.diff(timestamps)
    velocities = np.insert(velocities, 0, 0)  # 最初の点の速度は0とする

    in_fixation = False
    start_idx = 0
   
    for i in range(len(df)):
        if np.isnan(xs[i]) or np.isnan(ys[i]):
            if in_fixation:
                # 注視の終了
                in_fixation = False
                t_start = timestamps[start_idx]
                t_end = timestamps[i - 1]
                duration = (t_end - t_start) * 1000
                if duration >= duration_threshold_ms:
                    fixations.append({
                        "start_time": t_start,
                        "end_time": t_end,
                        "duration_ms": duration,
                        "x_mean_deg": np.mean(xs[start_idx:i]),
                        "y_mean_deg": np.mean(ys[start_idx:i]),
                    })
            continue

        if velocities[i] < velocity_threshold:
            if not in_fixation:
                in_fixation = True
                start_idx = i
        else:
            if in_fixation:
                in_fixation = False
                t_start = timestamps[start_idx]
                t_end = timestamps[i - 1]
                duration = (t_end - t_start) * 1000
                if duration >= duration_threshold_ms:
                    fixations.append({
                        "start_time": t_start,
                        "end_time": t_end,
                        "duration_ms": duration,
                        "x_mean_deg": np.mean(xs[start_idx:i]),
                        "y_mean_deg": np.mean(ys[start_idx:i]),
                    })

    # 最後が注視で終わっていた場合
    if in_fixation:
        t_start = timestamps[start_idx]
        t_end = timestamps[-1]
        duration = (t_end - t_start) * 1000
        if duration >= duration_threshold_ms:
            fixations.append({
                "start_time": t_start,
                "end_time": t_end,
                "duration_ms": duration,
                "x_mean_deg": np.mean(xs[start_idx:]),
                "y_mean_deg": np.mean(ys[start_idx:]),
            })

    return pd.DataFrame(fixations)


In [27]:
def deg_to_px(x_deg, y_deg):
    x_cm = np.tan(np.radians(x_deg)) * viewer_distance_cm
    y_cm = np.tan(np.radians(y_deg)) * viewer_distance_cm
    x_px = (x_cm / cm_per_pixel_x) + (monitor_resolution_px[0] / 2)
    y_px = (y_cm / cm_per_pixel_y) + (monitor_resolution_px[1] / 2)
    return x_px, y_px


In [ ]:
new1_df=interpolate_missing(eye_df)

new2_df=apply_gaussian_filter_by_block(new1_df)

detect_fixations_ivt(new2_df)


C:\Users\OWNER\AppData\Local\Temp\ipykernel_11072\2039497765.py:8: RuntimeWarning: divide by zero encountered in divide
  velocities = np.sqrt(np.diff(xs)**2 + np.diff(ys)**2) / np.diff(timestamps)


,start_time,end_time,duration_ms,x_mean_deg,y_mean_deg
0,1.732521e+09,1.732521e+09,344.000101,2.816316,13.541975
1,1.732521e+09,1.732521e+09,110.000134,1.039364,15.157164
2,1.732521e+09,1.732521e+09,125.000000,0.270768,15.010135
3,1.732521e+09,1.732521e+09,109.999895,1.006440,11.436087
4,1.732521e+09,1.732521e+09,187.000036,5.765666,14.652769
...,...,...,...,...,...
1147,1.732522e+09,1.732522e+09,111.999989,0.129844,13.240843
1148,1.732522e+09,1.732522e+09,142.999887,0.247377,13.649323
1149,1.732522e+09,1.732522e+09,143.000126,14.228503,-3.750319
1150,1.732522e+09,1.732522e+09,126.000166,2.150073,12.389878
